In [0]:
########## (a) ##########
# This code is used for JHU CS 482/682: Deep Learning 2019 Spring Homework 4
# Copyright @ Johns Hopkins University, Cong Gao, cgao11@jhu.edu
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST

# IMPORTANT: This function is used for de-normalizing image to original domain,
# please use this if you want to visualize/recover your output result
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

# These hyperparameters can be changed
num_epochs = 100
batch_size = 196 ## was 64
learning_rate = 1e-3

# This is a recommended normalization method. You can design your own

################
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

# Dataset wiill be downloaded to './data' folder
dataset = FashionMNIST('./data', transform=img_transform, download=True)
train_dataset = FashionMNIST('./data', train=True, transform=img_transform, download=True)
test_dataset = FashionMNIST('./data', train=False, transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True);
test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True);

In [0]:
########## (a) ##########
import torchvision.models as models
vgg16 = models.vgg16()
model = vgg16
model.classifier[6] = nn.Linear(4096, 10)
if torch.cuda.is_available(): #use gpu if available
    model.cuda()

In [3]:
########## (a) ##########
import time
import numpy as np


# This is a recommended optimizer setting
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()

# Training & Validation
for epoch in range(num_epochs):
    since = time.time()
    bar = 0
    for i, (images, labels) in enumerate(train_loader):

        images = Variable(images.float())
        labels = Variable(labels)

        images = torch.cat((images, images, images),1)
        
        optimizer.zero_grad()
        outputs = model(images.cuda())

        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()
        bar += 1
        if bar == 10:
            print('.', end=" ")
            bar = 0
    print("Epoch: %d" % (epoch+1))
    print("Training Loss: %.4f" % (loss.data.item()))
    ### Timing
    time_elapsed = time.time() - since
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Epoch: 1
Training Loss: 2.3025
3m 42s
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Epoch: 2
Training Loss: 2.3038
3m 41s
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Epoch: 3
Training Loss: 2.3023
3m 42s
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Epoch: 4
Training Loss: 2.3030
3m 42s


KeyboardInterrupt: ignored

In [4]:
########## (a) ##########
model.eval()
test_correct = 0
test_num = 0
for (images, labels) in test_loader:
    images = Variable(images.float())
    images = torch.cat((images, images, images),1)
    
    outputs = model(images.cuda())

    a, predicted = torch.max(outputs.data, 1)
    test_num += len(labels)
    test_correct += (predicted == labels.cuda()).sum().item()
print("Test Accuracy is: " + str(test_correct/float(test_num)*100) + "%")

Test Accuracy is: 10.0%


In [6]:
########## (b) ##########
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
model = vgg16
for param in model.parameters():
    param.requires_grad = False
model.classifier[6]=nn.Linear(4096, 10)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
553433881it [00:39, 13993270.92it/s]


In [9]:
########## (b) ##########
import time
import numpy as np
if torch.cuda.is_available(): #use gpu if available
    model.cuda()

# This is a recommended optimizer setting
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()

# Training & Validation
for epoch in range(num_epochs):
    since = time.time()
    losses=[]
    bar = 0

    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        images = torch.cat((images, images, images),1)
        
        optimizer.zero_grad()
        outputs = model(images.cuda())

        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data.item())
        bar += 1
        if bar == 10:
            print('.', end=" ")
            bar = 0
    mean_loss = sum(losses)/len(losses)
    print("Epoch: %d" % (epoch+1))
    print("Training Loss: %.4f" % (mean_loss))
    time_elapsed = time.time() - since
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

. . . . . . . . . Epoch: 1
Training Loss: 0.6915
0m 34s
. . . . . . . . . Epoch: 2
Training Loss: 0.6825
0m 34s
. . . . . . . . . Epoch: 3
Training Loss: 0.6841
0m 34s
. . . . . . . . . Epoch: 4
Training Loss: 0.6855
0m 34s
. . . . . . . . . Epoch: 5
Training Loss: 0.6787
0m 34s
. . . . . . . . . Epoch: 6
Training Loss: 0.6827
0m 34s
. . . . . . . . . Epoch: 7
Training Loss: 0.6761
0m 34s
. . . . . . . . . Epoch: 8
Training Loss: 0.6780
0m 34s
. . . . . . . . . Epoch: 9
Training Loss: 0.6751
0m 34s
. . . . . . . . . Epoch: 10
Training Loss: 0.6733
0m 34s
. . . . . . . . . Epoch: 11
Training Loss: 0.6785
0m 34s
. . . . . . . . . Epoch: 12
Training Loss: 0.6746
0m 34s
. . . . . . . . . Epoch: 13
Training Loss: 0.6820
0m 34s
. . . . . . . . . Epoch: 14
Training Loss: 0.6784
0m 35s
. . . . . . . . . Epoch: 15
Training Loss: 0.6764
0m 35s
. . . . . . . . . Epoch: 16
Training Loss: 0.6725
0m 35s
. . . . . . . . . Epoch: 17
Training Loss: 0.6745
0m 35s
. . . . . . . . . Epoch: 18
Training Los

KeyboardInterrupt: ignored

In [10]:
########## (b) ##########
model.eval()
test_correct = 0
test_num = 0
for (images, labels) in test_loader:
    images = Variable(images.float())
    images = torch.cat((images, images, images),1)
    
    outputs = model(images.cuda())

    a, predicted = torch.max(outputs.data, 1)
    test_num += len(labels)
    test_correct += (predicted == labels.cuda()).sum().item()
print("Test Accuracy is: " + str(test_correct/float(test_num)*100) + "%")

Test Accuracy is: 76.86%


In [0]:
########## (c) ##########
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
model = vgg16
model.features.requires_grad = False
model.avgpool.requires_grad = False
model.classifier[6] = nn.Linear(4096, 10)

In [15]:
########## (c) ##########
batch_size = 196 ## was 64

import time
import numpy as np
if torch.cuda.is_available(): #use gpu if available
    model.cuda()

# This is a recommended optimizer setting
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()

# Training & Validation
for epoch in range(num_epochs):
    since = time.time()
    losses=[]
    bar = 0

    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        images = torch.cat((images, images, images),1)
        
        optimizer.zero_grad()
        outputs = model(images.cuda())

        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data.item())
        bar += 1
        if bar == 10:
            print('.', end=" ")
            bar = 0
    mean_loss = sum(losses)/len(losses)
    print("Epoch: %d" % (epoch+1))
    print("Training Loss: %.4f" % (mean_loss))
    time_elapsed = time.time() - since
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

. . . . . . . . . Epoch: 1
Training Loss: 0.8685
3m 27s
. . . . . . . . . Epoch: 2
Training Loss: 0.2630
3m 27s
. . . . . . . . . Epoch: 3
Training Loss: 0.1578
3m 27s
. . . . . . . . . Epoch: 4
Training Loss: 0.1092
3m 27s
. . . . . . . . . Epoch: 5
Training Loss: 0.0818
3m 27s


KeyboardInterrupt: ignored

In [16]:
########## (c) ##########
model.eval()
test_correct = 0
test_num = 0
for (images, labels) in test_loader:
    images = Variable(images.float())
    images = torch.cat((images, images, images),1)
    
    outputs = model(images.cuda())

    a, predicted = torch.max(outputs.data, 1)
    test_num += len(labels)
    test_correct += (predicted == labels.cuda()).sum().item()
print("Test Accuracy is: " + str(test_correct/float(test_num)*100) + "%")

Test Accuracy is: 93.55%
